In [ ]:
from src.

class Encoder(object):
    def __init__(self):
        self.shape_predictor = 
        self.representation_predictor = 
    
    def encode(img):
        action = 
        args = None
        return action, args